In [1]:
#import libraries used into
from datetime import datetime, timedelta
import pandas as pd, numpy as np, os

In [33]:
##First, the vars was created
fileam = "OT " + datetime.now().strftime('%d.%m.%Y') + ".XLSX"
filepm = "OT " + datetime.now().strftime('%d.%m.%Y') + " 5 P.M..XLSX"
file3pm = "OT " + datetime.now().strftime('%d.%m.%Y') + " 3 P.M..XLSX"
fileyest = "OT " + (datetime.today() - timedelta(days = 1)).strftime('%d.%m.%Y' + " 5 P.M..XLSX")
main_path = "C:\\Users\\mruiz\\Documents\\DB OT" 
current_path = "C:\\Users\\mruiz\\Documents\\DB OT\\CurrentDocuments"
current_month = datetime.now().strftime('%B')
current_path_ = os.listdir(current_path)
main_path_ = os.listdir(main_path)
data_yesterday_excel = {}
data_am_excel = {}
data_3pm_excel = {}
data_pm_excel = {}

In [43]:
def get_data(entry_data):
    file = main_path + "\\" + entry_data
    filexcel = pd.read_excel(file,sheet_name="Sheet1",header=0)
    print("Los datos del archivo ", entry_data, " fueron procesados")
    return filexcel

In [53]:
while True:
    try:
        data_yesterday_excel = get_data(fileyest)
        data_am_excel = get_data(fileam)
        data_3pm_excel = get_data(file3pm)
        data_pm_excel = get_data(filepm)
    except FileNotFoundError:
        print("Hay al menos un archivo que no fue leído: ")
        print("Archivo Ayer : ", type(data_yesterday_excel))
        print("Archivo hoy por la mañana: ", type(data_am_excel))
        print("Archivo hoy 3 pm : ", type(data_3pm_excel))
        print("Archivo hoy 5 pm : ",type(data_pm_excel))
        break

Los datos del archivo  OT 25.10.2022 5 P.M..XLSX  fueron procesados
Los datos del archivo  OT 26.10.2022.XLSX  fueron procesados
Hay al menos un archivo que no fue leído: 
Archivo Ayer :  <class 'pandas.core.frame.DataFrame'>
Archivo hoy por la mañana:  <class 'pandas.core.frame.DataFrame'>
Archivo hoy 3 pm :  <class 'dict'>
Archivo hoy 5 pm :  <class 'pandas.core.frame.DataFrame'>


In [4]:
data2 =current_path + "\\Master_OT.xlsx"
data2excel = pd.read_excel(data2, sheet_name="Names", header=0)

In [5]:
data3 = current_path + "\\Politicas.XLSX"
data3excel = pd.read_excel(data3, sheet_name="POLITICAS CONTRATO", header=0)

In [6]:
data4 = current_path + "\\Materiales_Locales.xlsx"
data4excel = pd.read_excel(data4, sheet_name="ENTREGADO", header=0)

In [7]:
data5 = current_path + "\\Materiales_Foraneos.xlsx"
data5excel = pd.read_excel(data5, sheet_name="MATRIZ FINAL", header=1)

In [8]:
reprogramaciones = data_yesterday_excel.copy()

In [9]:
print(reprogramaciones)

{}


In [10]:
reprogramaciones.rename(columns={'Inicio programado':'Fecha_anterior'}, inplace=True)

AttributeError: 'dict' object has no attribute 'rename'

In [ ]:
if type(data_pm_excel) == type(data_yesterday_excel):
    data1excel = data_pm_excel
    print("Los datos de la tarde fueron seleccionados")
elif type(data_3pm_excel) == type(data_yesterday_excel):
    data1excel = data_3pm_excel
    print("Los datos del mediodia fueron seleccionados")
elif type(data_am_excel) == type(data_yesterday_excel):
    data1excel = data_am_excel
    print("Los datos de la mañana fueron seleccionados")

In [ ]:
data1excel.rename(columns={'Inicio programado' : 'Inicio_programado', 'Plan mant.preventivo' : 'Plan_Mtto',
       'Pto.tbjo.responsable': 'Responsable', 'Status del sistema' : 'Status', 'Texto breve' : 'Descripción',
       'Denominación de objeto técnico' : 'Cliente_Equipo', 'Número de serie': 'Número_de_serie', 'Clase de orden':'Clase_de_orden',
       'Fe.liberación real':'Fecha_Liberación', 'Fecha inicio real':'Fecha_creación'}, inplace = True)

In [ ]:
data1excel['Material'] = np.where((data1excel['Clase_de_orden'] == 'ZMCP') &  (data1excel['Descripción'].str.contains('Emer|Revi', case=False)), 'NO',
np.where(data1excel['Clase_de_orden'] == 'ZPM2','SI',
np.where(data1excel['Clase_de_orden'] == 'ZMCP','SI',
np.where(data1excel['Clase_de_orden'] == 'ZMCI', 'SI',
np.where((data1excel['Clase_de_orden'] == 'ZPM1') &  (data1excel['Descripción'].str.contains('CAFA|CABAT|ACUM', case=False)), 'SI','NO')))))

In [ ]:
#print('Las columnas de frame 1 son', data1excel.columns)
#print('Las columnas de frame 2 son', data2excel.columns)
#print('Las columnas de frame 3 son', data3excel.columns)
#print('Las columnas de frame 4 son', data4excel.columns)
#print('Las columnas de frame 5 son', data5excel.columns)
#print('Las columnas de reprogramaciones son ',reprogramaciones)

In [ ]:
result_df = pd.merge(left = data1excel, right = data2excel,left_on="Responsable",right_on="TecID",how="left").drop(columns=['TecID'])

In [ ]:
result_df = pd.merge(result_df,data3excel,on="Equipo",how="left")

In [ ]:
result_df = pd.merge(result_df,data4excel[['Orden', 'Inicio programado2', 'Fecha de salida',
       'Comentario', 'Material Separado']], on="Orden",how= "left")

In [ ]:
result_df = pd.merge(result_df,data5excel,on='Orden',how='left')

In [ ]:
result_df = pd.merge(left = result_df,right = reprogramaciones[['Orden','Fecha_anterior']],on='Orden',how='left')

In [ ]:
print(result_df.loc[0])

In [ ]:
result_df.assign(Pendiente_Recoger=0)

In [ ]:
result_df['Pendiente_Recoger'] = np.where(result_df['Inicio programado2'].isnull(),result_df['ESTATUS COMPRA'],result_df['Inicio programado2'])

In [ ]:
result_df.assign(reprogramacion=0)

In [ ]:
reprogramaciones = result_df.copy()

In [ ]:
result_df['reprogramacion'] = np.where(result_df['Inicio_programado']==reprogramaciones['Fecha_anterior'], "Sin_Reprogramar","Reprogramada")
result_df['reprogramacion'] = np.where(result_df['Fecha_anterior'].isnull(),"O.T. Agregada",result_df['reprogramacion'])

In [ ]:
print(result_df['reprogramacion'].unique())

In [ ]:
result_df.assign(Politica=0)

In [ ]:
result_df['Politica'] = np.where(result_df['Requiere Acceso'].isnull(),"NO","SI")

In [ ]:
print(result_df.columns)

In [ ]:
columnasdeseadas = ['Inicio_programado','Fecha_Liberación','Orden','Status','Material','Descripción', 'Short name','Estatus','City','Pendiente_Recoger', 'Fecha de salida', 'Comentario']
result_df2 = result_df[columnasdeseadas]

In [ ]:
"""columnasreprogramadas = ['Inicio_programado','Fecha_Liberación','Orden','Responsable','Status','Descripción','Fecha_anterior']
reprogramaciones_diarias = result_df[columnasreprogramadas]
path_repro = current_path + "\\Reprogramaciones.xlsx"
with pd.ExcelWriter(path_repro) as writer:
    reprogramaciones_diarias.to_excel(writer)"""


In [ ]:
os.remove(current_path + "\\Main.xlsx")
path = current_path + "\\Main.xlsx"
result_df.to_excel(path, index=False,header=True)
with pd.ExcelWriter(path) as writer:
    result_df.to_excel(writer,index=False,sheet_name="DB")
    result_df2.to_excel(writer,index=False ,sheet_name="Material")

In [ ]:
print(type(result_df))